## Hello everyone!

### In this kernel I am going to present some basic EDA for this dataset and show simple modeling for time series data.

<a id="top"></a>

<center><h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:Black; border:0' role="tab" aria-controls="home">Quick navigation</h3></center>

* [1. Data Analysis](#1)
* [2. Date feature engineering](#2)
* [3. Predict number of deaths until the end of 2021](#3)

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go
import plotly.express as px

import matplotlib.pyplot as plt

from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = pd.read_csv('/kaggle/input/us-police-shootings/shootings.csv')
df.head(10)

<a id="1"></a>
<center><h2 style='background:black; border:0; color:white'>Data Analysis<h2><center>

NaNs percent for every column.

In [ ]:
for col in df.columns:
    print(col, str(round(100* df[col].isnull().sum() / len(df), 2)) + '%')

In [ ]:
data = df['manner_of_death'].value_counts().reset_index()

data.columns = [
    'manner_of_death', 
    'count'
]

fig = px.pie(
    data, 
    values='count', 
    names='manner_of_death', 
    title='Manner of death', 
    width=500, 
    height=500
)

fig.show()

In [ ]:
data = df['armed'].value_counts().reset_index()

data.columns = [
    'armed', 
    'count'
]

data = data.sort_values('count')

fig = px.bar(
    data.tail(25), 
    x='count', 
    y='armed', 
    orientation='h', 
    title='Weapon', 
    width=800, 
    height=800
)

fig.show()

We can see that the most popular weapons are gun and knife.

In [ ]:
fig = px.histogram(
    df, 
    "age", 
    nbins=80, 
    title ='Age distribution', 
    width=800,
    height=500
)

fig.show()

In [ ]:
fig = go.Figure(
    data=go.Violin(
        y=df['age'], 
        x0='Age'
    )
)

fig.show()

In [ ]:
data = df['gender'].value_counts().reset_index()

data.columns = [
    'gender', 
    'count'
]

fig = px.pie(
    data, 
    values='count', 
    names='gender',  
    title='Gender distribution', 
    width=500, 
    height=500
)

fig.show()

In [ ]:
data = df['race'].value_counts().reset_index()

data.columns = [
    'race', 
    'count'
]

data = data.sort_values('count')

fig = px.bar(
    data, 
    x='count', 
    y='race', 
    orientation='h', 
    title='Race distribution', 
    width=600,
    height=600
)

fig.show()

In [ ]:
city = df.groupby('city')['name'].count().reset_index().sort_values('name', ascending=True).tail(50)

fig = px.bar(
    city, 
    x="name", 
    y="city", 
    orientation='h',
    title="Top 50 cities by deaths", 
    width=800, 
    height=900
)

fig.show()

<a id="2"></a>
<center><h2 style='background:black; border:0; color:white'>Date feature engineering<h2><center>

### Lets extract some standard features from date and check them

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df["weekday"] = df["date"].dt.weekday
df['month'] = df['date'].dt.month
df['month_day'] = df['date'].dt.day
df['year'] = df['date'].dt.year

In [ ]:
data = df.groupby(['weekday'])['name'].count().reset_index()

data.columns = [
    'weekday', 
    'count'
]

fig = px.bar(
    data, 
    x='weekday', 
    y='count',
    orientation='v', 
    title='Day of week', 
    width=600
)

fig.show()

In [ ]:
data = df.groupby(['month'])['name'].count().reset_index()

data.columns = [
    'month', 
    'count'
]

fig = px.bar(
    data, 
    x='month', 
    y='count', 
    orientation='v', 
    title='Month of year', 
    width=800
)

fig.show()

In [ ]:
data = df.groupby(['month_day'])['name'].count().reset_index()

data.columns = [
    'month_day', 
    'count'
]

fig = px.bar(
    data, 
    x='month_day', 
    y='count', 
    orientation='v', 
    title='Day of month', 
    width=800
)

fig.show()

In [ ]:
data = df.groupby(['year'])['name'].count().reset_index()

data.columns = [
    'year', 
    'count'
]

fig = px.bar(
    data, 
    x='year', 
    y='count',
    orientation='v', 
    title='Year', 
    width=600
)

fig.show()

In [ ]:
df['month_year'] = pd.to_datetime(df['date']).dt.to_period('M')
df.head()

In [ ]:
data = df.groupby(['month_year'])['name'].count().reset_index()

data.columns = [
    'month_year', 
    'count'
]

data['month_year'] = data['month_year'].astype(str)

fig = px.bar(
    data, 
    x='month_year', 
    y='count', 
    orientation='v', 
    title='Deaths by months', 
    width=800
)

fig.show()

In [ ]:
data = df.groupby(['flee'])['name'].count().reset_index()

data.columns = [
    'flee', 
    'count'
]

fig = px.bar(
    data, 
    x='flee', 
    y='count', 
    orientation='v', 
    title='Flee distribution', 
    width=600
)
fig.show()

<a id="3"></a>
<center><h2 style='background:black; border:0; color:white'>Predict number of deaths until the end of 2021<h2><center>

### We will use simple ARIMA model for forecasting

In [ ]:
data = df.groupby(['month_year'])['name'].count().reset_index()

data.columns = [
    'month_year', 
    'count'
]

data['month_year'] = data['month_year'].astype(str)
data = data.head(65)

fig = px.line(
    data, 
    x="month_year", 
    y="count", 
    title='Deaths month by month'
)

fig.show()

In [ ]:
model = ARIMA(
    data['count'], 
    order=(3,1,1)
)
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
preds, _, _ = model_fit.forecast(
    19, 
    alpha=0.05
)
preds = preds.tolist()
preds = [
    int(item) for item in preds
]

months = [
    '2020-06', '2020-07', '2020-08', 
    '2020-09', '2020-10', '2020-11', 
    '2020-12', '2021-01', '2021-02', 
    '2021-03', '2021-04', '2021-05', 
    '2021-06', '2021-07', '2021-08', 
    '2021-09', '2021-10', '2021-11', 
    '2021-12'
]

new_df = pd.DataFrame()
new_df['month_year'] = months
new_df['count'] = preds
data = pd.concat([data, new_df])

In [ ]:
fig = px.line(
    data, 
    x="month_year", 
    y="count", 
    title='Deaths month by month with predictions'
)

fig.show()